## Center of small quantum group

This Jupyter notebook shows how to use this program to compute the center of the small quantum group. This is under devolpment, but otherwise fully functional. We think that in the future this will be much faster, at which point this notebook will become more detailed.

In [1]:
# Importing the necessary modules.

import sys
sys.path.append('../bggcomplex')

from bggcomplex import BGGComplex
from quantum_center_module import QuantumFactory

In [2]:
BGG = BGGComplex('G2')

In [3]:
print('The set of simple roots is: %s' % range(1,len(BGG.simple_roots)+1))

The set of simple roots is: [1, 2]


In [4]:
q_factory = QuantumFactory(BGG,simple_root_subset=set([1]))

In [5]:
q_factory.compute_cohomology_block()

(2, 0)
made a weight module of size: 5, with 0 relations)
(2, 0, [(0, 1)])
(4, 0)
made a weight module of size: 10, with 0 relations)
(4, 0, [(0, 1)])
(6, 0)
made a weight module of size: 10, with 0 relations)
(6, 0, [(0, 1)])
(8, 0)
made a weight module of size: 5, with 0 relations)
(8, 0, [(0, 1)])
(10, 0)
made a weight module of size: 1, with 0 relations)
(10, 0, [(0, 1)])
(4, 2)
made a weight module of size: 190, with 90 relations)
(4, 2, [(0, 1)])
(6, 2)
made a weight module of size: 165, with 90 relations)
(6, 2, [(2*alpha[1] + alpha[2], 1), (0, 1)])
(8, 2)
made a weight module of size: 75, with 45 relations)
(8, 2, [(2*alpha[1] + alpha[2], 1), (0, 1)])
(6, 4)
made a weight module of size: 1275, with 1449 relations)
(6, 4, [])


{(2, 0): [(0, 1)],
 (4, 0): [(0, 1)],
 (4, 2): [(0, 1)],
 (6, 0): [(0, 1)],
 (6, 2): [(2*alpha[1] + alpha[2], 1), (0, 1)],
 (6, 4): [],
 (8, 0): [(0, 1)],
 (8, 2): [(2*alpha[1] + alpha[2], 1), (0, 1)],
 (10, 0): [(0, 1)]}